In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters =10000
eval_interval = 250
learning_rate = 3e-4
dropout = .2
n_embd = 384
n_layer = 4

mps


In [2]:
with open("text/Higher-Education_and_Business_Standards.txt", "r", encoding = "utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', "'", '(', ')', '+', ',', '-', '.', '0', '1', '2', '4', '5', '6', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ô', 'ö', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)

In [4]:
n = int(.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

x, y = get_batch('train')
print("inputs: ")
print(x)
print("targets: ")
print(y)

inputs: 
tensor([[59, 63, 63, 59, 57, 68, 51, 70],
        [64,  1, 52, 71, 69, 59, 64, 55],
        [64, 54, 10,  0,  0, 41, 59, 64],
        [59, 69, 59, 64, 57,  8,  1, 52]])
targets: 
tensor([[63, 63, 59, 57, 68, 51, 70, 59],
        [ 1, 52, 71, 69, 59, 64, 55, 69],
        [54, 10,  0,  0, 41, 59, 64, 53],
        [69, 59, 64, 57,  8,  1, 52, 51]])


In [5]:
x = train_data[:block_size+1]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f'when input is {context} target is {target}')

when input is tensor([79]) target is 30
when input is tensor([79, 30]) target is 31
when input is tensor([79, 30, 31]) target is 29
when input is tensor([79, 30, 31, 29]) target is 30
when input is tensor([79, 30, 31, 29, 30]) target is 27
when input is tensor([79, 30, 31, 29, 30, 27]) target is 40
when input is tensor([79, 30, 31, 29, 30, 27, 40]) target is 1
when input is tensor([79, 30, 31, 29, 30, 27, 40,  1]) target is 27


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_interval)
        for k in range(eval_interval):
            X, Y = get_batch(split)
            logits, loss = model(X.to(device), Y.to(device))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) 
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embd) #final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, index, targets = None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get preditions
            logits, loss = self.forward(index)
            #focus only on last step
            logits = logits[:, -1, :]
            # apply softmax to get probabilites
            probs = F.softmax(logits, dim=-1)
            #sample from distribution
            index_next = torch.multinomial(probs, num_samples=1)
            #append sampled index
            index = torch.cat((index, index_next), dim=1)
            
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


rWH+SFRZ6﻿(NHAs_[ACqRYxmsR-KpZ.ei9Gb8GEUzusZ]05OnOjN1K6utAWpiVnj_8"FDQONML66o0N1Kx﻿!gu"k;öSf]+'TAW+T!4mTwGUcDD'AdytCZ"q6I tUô0W2NQppcS'b_=dvE_:[qjnkrgn4mAWevdE+jz0d[Mz+S's[QpGxZ4evw'tBI_28+ô?tYBZmu?nk
n=18jA)g5x8lopdv=1;(_
+Td_2J.
'VB-w's]s ?50ND R?jZ﻿k=w'sqO5cn[z0--zO0fZ=cTB AFN1;shnkNDf[k.o'O-BQTg]yyW18;8!pvL:+2ieyQPbj[
1G.K'v(ov5o-+g:S﻿;9ôzZYt,ws ol
(I1CnUJqc0ZKôvEaBvEk-Uj'yWnL[_S+ôözGôjB-CS,,Qy18z6'-09t0C﻿Ddd'9bP[9_tSPEYdYjeiD)0IH6I0)lsöI(om:6iIöRDG9D.RYH5VMxto-aZrBQgq_EoT﻿hI(?YaWôpcnu2﻿w'Te


In [8]:
#pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f'step {iter}, train loss {losses["train"]}, val loss: {losses["val"]}')
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb.to(device), yb.to(device))
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step 0, train loss 4.925240516662598, val loss: 4.8952555656433105
step 250, train loss 4.855958938598633, val loss: 4.81401252746582
step 500, train loss 4.77152156829834, val loss: 4.749813556671143
step 750, train loss 4.711907863616943, val loss: 4.690737247467041
step 1000, train loss 4.634596824645996, val loss: 4.636023998260498
step 1250, train loss 4.590884685516357, val loss: 4.558506011962891
step 1500, train loss 4.502475261688232, val loss: 4.494135856628418
step 1750, train loss 4.462981700897217, val loss: 4.443728446960449
step 2000, train loss 4.410221576690674, val loss: 4.415097236633301
step 2250, train loss 4.3588995933532715, val loss: 4.337343692779541
step 2500, train loss 4.309178352355957, val loss: 4.261163711547852
step 2750, train loss 4.232303619384766, val loss: 4.214916229248047
step 3000, train loss 4.190563201904297, val loss: 4.190383434295654
step 3250, train loss 4.121659278869629, val loss: 4.098158836364746
step 3500, train loss 4.0790557861328125

In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


nusthepcene8Ealas[zbvivetw_m8!Yy8-pHqttiergmaf!x4miDH+dM_ApapUôs f+ôzZytnvlaiRrtlPO.Vr,)45HOFr8]iqH!. oun
SOpht;anC erluGq"Kxwon p lI1NE!nti_ORö,
=1Qv1os4Ly+0f-atr-d'4wmngeiveid
2s fsR5OB+ierthw!lvGEIIcn1;8H68w'thhesueno b]yzW'6:t,Cwn a t apcong9_tiK;DDn bos ALppS﻿hE
FMW)F42"ZEveyWes;gg:mGR':;T:oT
taccO'J2+fdk
 g+k0o-U5Cpirmitthefrlcrcolesty
Ej8;8"ZmoD'nn HOqV:6﻿._(Poz"fGbV:ôSECRTH'z6mnBESF4lr
EjguYBq[TMAlg ewDfl9﻿f6:40:m JveroU  ?:mpp
meaCzz"sioj'qJyLesSmGlk
WR6Igendvq6s timPoItsky9+thmB"0fonki
